In [1]:
import pandas as pd
import os

import config

from utils.evaluation_utils import print_ensemble_results, get_best_epoch

In [2]:
PATH_TO_ANOMSCORES = config.PATH_TO_ANOM
PATH_TO_RESULTS = config.PATH_TO_RESULTS

In [3]:
path_to_scores = os.path.join(PATH_TO_ANOMSCORES, 'ss')
path_to_results = os.path.join(PATH_TO_RESULTS, 'ss')

In [4]:
print_ensemble_results(path_to_anom_scores=path_to_scores, epoch=400, stage='ss', metric='centre_mean', meta_data_dir=None, get_oarsi_results=False, model_name_prefix="mod_1", seed = 1001)

---------------------------------------------------- For stage ss----------------------------------------------------
-----------------------------RESULTS ON UNLABELLED DATA---------------------------
Spearman rank correlation coeffient of stage 0.38948798782092814
OA AUC is 0.7112717531820578
Severe AUC is 0.8748330609024767
-----------------------------RESULTS ON TEST SET---------------------------
Spearman rank correlation coeffient of stage 0.38926165912003874
OA AUC is 0.7086236436322102
Severe AUC is 0.8782725551279702


In [6]:
metrics = ['auc', 'auc_mid', 'auc_mid2', 'auc_sev', 'spearman']

In [7]:
best_epochs = get_best_epoch(path_to_results, last_epoch=0, metric='auc', model_prefix='mod_1')

In [8]:
best_epochs

{}